In [1]:
#!/usr/bin/env python
# coding: utf-8

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 10,10

from scipy.ndimage.interpolation import zoom
import sys
sys.path.insert(0,'..')
from Classifier import ResNet
from utils import gradcamutils
import numpy as np
import os
from PIL import Image
import innvestigate

Using TensorFlow backend.
C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hp\Anaconda3\envs\wbvenv36\lib\site-pa

In [2]:
# use this environment flag to change which GPU to use 
os.environ["CUDA_VISIBLE_DEVICES"]= "-1"  # specify which GPU(s) to be used

In [3]:
def createOrRestoreModel(): 
    resnet = ResNet.ResNet18((224,224,3),4) #, False)

    resnet.summary()
    resnet.load_weights("ResNet18_COVID19.h5") #load weights

    model = resnet

    return model 

In [4]:
def readTestImage(img_path): 
    img = Image.open(img_path) #open image you want to visualize "1238_R.png"

    img = np.array(img.resize((224,224), Image.ANTIALIAS))
    imgArr = img.reshape(1,224,224,3)#open image you want to visualize
    imgNormalized = imgArr / 255.
    
    return img, imgArr, imgNormalized

In [5]:
img_path = "../Example_images/1312A392-67A3-4EBF-9319-810CF6DA5EF6.jpeg"
img, imgArr, imgNormalized = readTestImage(img_path)

In [6]:
imgNormalized.shape

(1, 224, 224, 3)

In [7]:
model = createOrRestoreModel() 


reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           b

In [8]:
def createLRPAnalyzer(model, imgNormalized):
    '''
    All possible names are: ['input', 'random', 'gradient', 'gradient.baseline', 'input_t_gradient', 
    'deconvnet', 'guided_backprop', 'integrated_gradients', 'smoothgrad', 'lrp', 'lrp.z', 'lrp.z_IB', 
    'lrp.epsilon', 'lrp.epsilon_IB', 'lrp.w_square', 'lrp.flat', 'lrp.alpha_beta', 'lrp.alpha_2_beta_1', 
    'lrp.alpha_2_beta_1_IB', 'lrp.alpha_1_beta_0', 'lrp.alpha_1_beta_0_IB', 'lrp.z_plus', 'lrp.z_plus_fast', 
    'lrp.sequential_preset_a', 'lrp.sequential_preset_b', 'lrp.sequential_preset_a_flat', 
    'lrp.sequential_preset_b_flat', 'deep_taylor', 'deep_taylor.bounded', 'deep_lift.wrapper', 
    'pattern.net', 'pattern.attribution']"
    '''

    analyzer = innvestigate.create_analyzer("lrp.sequential_preset_a", model, reverse_keep_tensors=True)
    analysis = analyzer.analyze(imgNormalized)
    
    relScore = analyzer._reversed_tensors
    
    a = analysis
    a /= np.max(np.abs(a))
    
    return a, relScore

In [9]:
gradCAM = gradcamutils.grad_cam(model, imgNormalized, layer_name = 'conv2d_20') #for VGG, here there are parameters to set image width (W) and height (H)
gradCAMPlus = gradcamutils.grad_cam_plus(model, imgNormalized, layer_name = 'conv2d_20') # conv2d_120 -> conv2d_20 
LRP, relScore = createLRPAnalyzer(model, imgNormalized)


1
1


NotAnalyzeableModelException: This analysis method does not support softmax layers.
Check triggerd by layers: [<keras.layers.core.Dense object at 0x0000020A1D6859E8>]

In [ ]:
pred = model.predict(imgNormalized)

class_prob = pred.tolist()
softMaxProb = class_prob[0]

In [ ]:
for score in relScore:
    print(score)

In [ ]:
def remove_ticks_and_labels(ax):
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.tick_params(axis='both', which='both', length=0)

In [ ]:
def decisionVisualization(img, xaiMethod, vizMethod):
    # Add batch axis and preprocess   

    fig = plt.figure(constrained_layout=False,figsize=[6,3])
    gs1 = fig.add_gridspec(nrows=1, ncols=2, left=1, right=1.8)

    fig_ax1 = fig.add_subplot(gs1[0, 0])
    fig_ax2 = fig.add_subplot(gs1[0, 1])

    gs2 = fig.add_gridspec(nrows=1, ncols=1, left=1.92, right=2.6)

    fig_ax3 = fig.add_subplot(gs2[0, 0])

    fig_ax1.imshow(img, cmap ='gray')
    fig_ax1.set_title("Input radiograph")

    remove_ticks_and_labels(fig_ax1)
    
    if vizMethod == 'LRP':
        norm = LRP[0,:,:,0] 

        fig_ax2.imshow(img, cmap ='Spectral')
        fig_ax2.imshow(norm, vmax = 1, vmin = -1,alpha=0.45, cmap="jet")
        fig_ax2.set_title(str(vizMethod))
        remove_ticks_and_labels(fig_ax2)

        #x_probs = [0.12, 0.30, 0.48, 0.10]
        x_probs = softMaxProb
        
        x_probs = np.asarray(x_probs, dtype=np.float32)
        y_pos = np.array([0, 1, 2, 3])
        y_objects = ('Type-1','Type-2','Type-3','Type-4') #Infection type
        y_labels = ['Normal', 'Bacterial', 'Non-COVID19 Viral', 'COVID-19 Viral']

        fig_ax3.barh(y_pos, x_probs, color='green', align='center', alpha=0.3)
        fig_ax3.set_title("Explanations")
    
        for i, v in enumerate(x_probs):
            fig_ax3.text(v - 0.07, i + 0.1 , '{0:.2f}   {1}'.format(v,y_labels[i]))
    
        fig_ax3.set_yticks(y_pos)
        fig_ax3.set_yticklabels(y_objects)
        fig_ax3.invert_yaxis()  # labels read top-to-bottom
        fig_ax3.set_xticks([0, 0.25, 0.5, 0.75])

        fig.savefig('explanation.png')
        
    if vizMethod == 'CAM':

        fig_ax2.imshow(img, cmap ='gray')
        fig_ax2.imshow(xaiMethod, alpha=0.45, cmap="jet")
        fig_ax2.set_title(str(vizMethod))
        remove_ticks_and_labels(fig_ax2)

        #x_probs = [0.12, 0.30, 0.48, 0.10]
        x_probs = softMaxProb
        x_probs = np.asarray(x_probs, dtype=np.float32)
        y_pos = np.array([0, 1, 2, 3])
        y_objects = ('Type-1','Type-2','Type-3','Type-4') #Infection type
        y_labels = ['Normal', 'Bacterial', 'Non-COVID19 Viral', 'COVID-19 Viral']

        fig_ax3.barh(y_pos, x_probs, color='blue', align='center', alpha=0.3)
        fig_ax3.set_title("Explanations")
    
        for i, v in enumerate(x_probs):
            fig_ax3.text(v - 0.07, i + 0.1 , '{0:.2f}   {1}'.format(v,y_labels[i]))
    
        fig_ax3.set_yticks(y_pos)
        fig_ax3.set_yticklabels(y_objects)
        fig_ax3.invert_yaxis()  # labels read top-to-bottom
        fig_ax3.set_xticks([0, 0.25, 0.5, 0.75])

        fig.savefig('explanation.png')
        
    if vizMethod == 'CAMPlus':

        fig_ax2.imshow(img, cmap ='gray')
        fig_ax2.imshow(xaiMethod, alpha=0.45, cmap="jet")
        fig_ax2.set_title(str(vizMethod))
        remove_ticks_and_labels(fig_ax2)

        #x_probs = [0.12, 0.30, 0.48, 0.10]
        x_probs = softMaxProb
        x_probs = np.asarray(x_probs, dtype=np.float32)
        y_pos = np.array([0, 1, 2, 3])
        y_objects = ('Type-1','Type-2','Type-3','Type-4') #Infection type
        y_labels = ['Normal', 'Bacterial', 'Non-COVID19 Viral', 'COVID-19 Viral']

        fig_ax3.barh(y_pos, x_probs, color='red', align='center', alpha=0.3)
        fig_ax3.set_title("Explanations")
    
        for i, v in enumerate(x_probs):
            fig_ax3.text(v - 0.07, i + 0.1 , '{0:.2f}   {1}'.format(v,y_labels[i]))
    
        fig_ax3.set_yticks(y_pos)
        fig_ax3.set_yticklabels(y_objects)
        fig_ax3.invert_yaxis()  # labels read top-to-bottom
        fig_ax3.set_xticks([0, 0.25, 0.5, 0.75])

        fig.savefig('explanation.png')

In [ ]:
decisionVisualization(img, xaiMethod=LRP, vizMethod='LRP')

In [ ]:
decisionVisualization(img, xaiMethod=gradCAM, vizMethod='CAM')

In [ ]:
decisionVisualization(img, xaiMethod=gradCAMPlus, vizMethod='CAMPlus')